In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table
from astropy.time import Time

import ipywidgets as ipw

import matplotlib.pyplot as plt

import numpy as np

from stellarphot.differential_photometry.aij_rel_fluxes import calc_aij_relative_flux
from stellarphot.settings.fits_opener import FitsOpener
from stellarphot import PhotometryData, SourceListData

### Choose photometry and aperture files

In [ ]:
fits_openr = FitsOpener(title="Select your photometry file", filter_pattern=["*.csv", "*.ecsv"])
fits_openr2 = FitsOpener(
    title="Select your aperture file", filter_pattern=["*.csv", "*.fits", "*.ecsv"]
)

In [ ]:
box = ipw.VBox()
box.children = [fits_openr.file_chooser, fits_openr2.file_chooser]
box

### Run the remaining cells

In [ ]:
aperture_file = fits_openr2.path
photometry_file = fits_openr.path

output_file = photometry_file.stem + "-relative-flux" + photometry_file.suffix

In [ ]:
photometry = PhotometryData.read(photometry_file)

In [ ]:
comp_table = SourceListData.read(aperture_file)

In [ ]:
comp_table["coord"] = SkyCoord(
    ra=comp_table["ra"], dec=comp_table["dec"], frame="icrs"
)
comp_bool = comp_table["marker name"] == ["APASS comparison"]
only_comp_stars = comp_table[comp_bool]
coords = only_comp_stars["coord"]

### Calculate flux

In [ ]:
# calculate flux
flux_table = calc_aij_relative_flux(photometry, only_comp_stars)

In [ ]:
if "bjd" not in flux_table.colnames:
    flux_group = flux_table.group_by("file")
    # Add dummy BJD column so the whole table has one
    flux["bjd"] = np.nan
    for group in flux_group.groups:
        mean_ra = group["ra"].mean()
        mean_dec = group["dec"].mean()
        group.add_bjd_col(bjd_coordinates=SkyCoord(mean_ra, mean_dec))

In [ ]:
flux_table.write(output_file, overwrite=True)